In [ ]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash;

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl;
from dash import dcc, html, dash_table, ctx;
import plotly.express as px;
from dash.dependencies import Input, Output, State;
import base64;

# Configure OS routines
import os;

# Configure the plotting routines
import numpy as np;
import pandas as pd;
import matplotlib.pyplot as plt;

# Custom Class
from CRUD import CRUD;

###########################
# Data Manipulation / Model
###########################
username = "";
password = "";
host = "";
port = 0;
db = "";
collection = "";

# Connect to database via CRUD Module
db = CRUD(username, password, host, port, db, collection);

df = pd.DataFrame.from_records(db.read({}));

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True);

## Debug
# print(len(df.to_dict(orient='records')));
# print(df.columns);

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__);

image_filename = 'Grazioso_Salvare_Logo.png';
encoded_image = base64.b64encode(open(image_filename, 'rb').read());

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    
    #Custom dashoard header - DT
    html.Div([
    html.A([
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={"width":200, "height":200})
    ], href="https://www.snhu.edu", target="_blank"),
    html.H1("Welcome to the CS-340 Dashboard", style={"width":"50%", "margin": "auto"})
    ], style={"display": "flex", "width":"100%"}),
    
    html.H3("Developer: https://github.com/DaultonTruett",
                style={"text-align": "right"}),
    html.Hr(),
    
    #Buttons to enable users to query DB for breeds for specific training - DT
    html.Div(className="buttonRow", style={"display": "flex"},
             children=[
                 html.H3("Filter table to search breeds for training: "),
                 html.Button(id="water_btn", style={"margin": "1px 16px"},
                             children=html.H4("Water Rescue")),
                 html.Button(id="mtn_btn", style={"margin": "1px 16px"},
                             children=html.H4("Mountain/Wilderness Rescue")),
                 html.Button(id="disaster_btn", style={"margin": "1px 16px"},
                             children=html.H4("Disaster/Individiual Tracking")),
                 html.Button(id="reset_btn", style={"margin": "1px 16px"},
                             children=html.H4("Reset Table"))
             ]),
    
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        row_selectable="single",
        selected_rows=[0],
        selected_columns = [],
        filter_action="native",
        sort_action="native",
        page_action="native",
        page_current=0,
        page_size=10,
        style_header={
            "backgroundColor": "rgb(210, 210, 210)" ,
            "color": "black",
            "fontWeight": "bold"
        },
        style_data={
            "whiteSpace": "normal",
            "height": "auto",
            "color": "black",
            "backgroundColor": "white"
        },
        style_data_conditional=[
            {
                "if": {"row_index": "odd"},
                "backgroundColor": "rgb(220, 220, 220)"
            }
        ],
        style_table={
            "overflowX": "auto",
            "width": "100%"
        },
        style_cell={
            "textAlign": "left",
            "height": "auto",
            "width": "auto",
            "whiteSpace": "normal"
        },                    
    ),
    
    html.Br(),
    html.Hr(),
    
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex', "margin":"20px", "width": "100%"},
             children=[
        html.Div(
            id='map-id',
            className='col s12 m6',
        ),
        html.Div(
            className='col s12 m6',
            children=[
            html.Center(html.H2(id="graph-title")),
            dcc.Graph(id='graph-id'),
        ]),
             ]),
], style={"margin": "20px"});


#############################################
# Interaction Between Components / Controller
#############################################

#Callback for the button menu to filter DB for dog breeds based on desired training capabilities
#Utilizes a compound query to return desired breeds that are within an age range and a particular sex outcome - DT
@app.callback(
    Output('datatable-id','data'),
    [Input("water_btn", "n_clicks"),
     Input("mtn_btn", "n_clicks"),
     Input("disaster_btn", "n_clicks"),
     Input('reset_btn', 'n_clicks')
    ])
def on_click(btn_one, btn_two, btn_three, btn_four):
    df = pd.DataFrame.from_records(db.read({}));
    triggered_id = ctx.triggered_id;
    
    if triggered_id == "water_btn":
        water_rescue = {
            "$and": [
                {
                    "breed": {"$in": ["Labrador Retriever Mix", "Chesa Bay Retr Mix", "Newfoundland Mix"]},
                    "age_upon_outcome_in_weeks": {"$gte": 26},
                    "sex_upon_outcome": "Intact Female"
                },
                {
                    "breed": {"$in": ["Labrador Retriever Mix", "Chesa Bay Retr Mix", "Newfoundland Mix"]},
                    "age_upon_outcome_in_weeks": {"$lte": 156},
                    "sex_upon_outcome": "Intact Female"
                },              
                ]
            };
        df = pd.DataFrame.from_records(db.read(water_rescue));
        
    elif triggered_id == "mtn_btn":
        mtn_rescue = {
            "$and": [
                {
                    "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog, Siberian Husky",
                                     "Rottweiler"]},
                    "age_upon_outcome_in_weeks": {"$gte": 26},
                    "sex_upon_outcome": "Intact Male"
                },
                {
                    "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog, Siberian Husky",
                                     "Rottweiler"]},
                    "age_upon_outcome_in_weeks": {"$lte": 156},
                    "sex_upon_outcome": "Intact Male"
                },              
                ]
            };
        df = pd.DataFrame.from_records(db.read(mtn_rescue));
        
    elif triggered_id == "disaster_btn":
        tracking = {
            "$and": [
                {
                    "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound",
                                     "Rottweiler"]},
                    "age_upon_outcome_in_weeks": {"$gte": 20},
                    "sex_upon_outcome": "Intact Male"
                },
                {
                    "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound",
                                     "Rottweiler"]},
                    "age_upon_outcome_in_weeks": {"$lte": 300},
                    "sex_upon_outcome": "Intact Male"
                },              
                ]
            };
        df = pd.DataFrame.from_records(db.read(tracking));
        
    elif triggered_id == "reset_btn":
        df = pd.DataFrame.from_records(db.read({}));

    df.drop(columns=['_id'],inplace=True);
    
    return df.to_dict('records');


#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')
    ])
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns];


#This callback allows the selected row to always default to the 0th index even after a user
#selects a row and changes pages and/or refreshes the datatable with a filter query
@app.callback(
    Output("datatable-id", "selected_rows"),
    [Input("datatable-id", "page_current")
    ])
def update_selected_rows(data):
    new_selected_row = [0];
    return new_selected_row;


#Pass in the new data to the pie-chart from the datatable as filter options return a new dataset
#The pie-chart will reflect all breeds that are currently populating the datatable - DT
@app.callback(
    Output('graph-id', "figure"),
    [Input('datatable-id', "derived_virtual_data")
    ])
def update_graphs(viewData):
    if viewData is None:
        return;

    df = pd.DataFrame.from_dict(viewData);
    figure = px.pie(df, names="breed", width = 700, height = 600);
    
    figure.update_layout(uniformtext_minsize=12, uniformtext_mode="hide");
    figure.update_traces(textposition="inside", textinfo="value");

    return figure;


#Dynamically updates the pie-chart to reflect the 
#breeds that correspond to a particular training  - DT
@app.callback(
    Output("graph-title", "children"),
    [Input("reset_btn", "n_clicks"),
     Input("water_btn", "n_clicks"),
     Input("mtn_btn", "n_clicks"),
     Input("disaster_btn", "n_clicks")
    ])
def update_graph_title(btn1, btn2, btn3, btn4):
    triggered_id = ctx.triggered_id;

    if triggered_id == "reset_btn":
        title="All breeds";
 
    elif triggered_id == "water_btn":
        title="Water Rescue Breeds";
        
    elif triggered_id == "mtn_btn":
        title="Mountain/Wilderness Rescue Breeds";
        
    elif triggered_id == "disaster_btn":
        title="Disaster/Individual Tracking Breeds";
        
    else:
        title = "All Breeds";
        
    return title;


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")
    ])
def update_map(viewData, index):
    dff = pd.DataFrame.from_dict(viewData);
    
    #If no data is available, default to the Austin Animal Shelter
    if viewData is None:
        markerCoords = [30.75, -97.48];
        Tooltip = "Austin Animal Shelter";
        Popup = "Austin Animal Shelter";
    
    #Tailor the map data to the particualr dog chosen in the datatable
    else:
        row = index[0];
        markerCoords = [dff.iloc[row, 13], dff.iloc[row, 14]];
        Tooltip = dff.iloc[row, 4];
        Popup = dff.iloc[row, 9];

    map = dl.Map(
        style = {"width": "600px","height": "600px"}, center = markerCoords, zoom = 10,
        children=[dl.TileLayer(id="base-layer-id"), dl.Marker(position = markerCoords,
            children=[dl.Tooltip(Tooltip), dl.Popup([html.H1("Animal name"), html.P(Popup)])])]);
    
    return map;


app.run_server(debug=True);
